### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220711_scenario_projections.ipynb")

2022/07/13 20:48:30 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220628_july_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/07/13 20:48:33 INFO|-------------------------|{} Retrieving vaccinations data
2022/07/13 20:48:35 INFO|-------------------------|{} Constructing vaccination projections
2022/07/13 20:48:35 INFO|-------------------------|{} Retrieving hospitalizations data
2022/07/13 20:48:37 INFO|----------------------|{} Prepping Model
2022/07/13 20:48:37 INFO|-----------------------|{} Prepping Model
2022/07/13 20:49:29 INFO|----------------------|{} Model prepped for fitting in 52.22339170800001 seconds.
2022/07/13 20:49:29 INFO|----------------------|{} Will fit 25 times
2022/07/13 20:50:31 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 62.19850191699999 seconds: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5380334941579741}, 42: {'co': 0.0005409486889829486}, 56: {'co': 0.6816118768777373}, 70: {'co': 0.89119421452826}}
2022/07/13 20:50:31 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/07/13

2022/07/13 20:54:28 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 10.136924915999998 seconds: {504: {'co': 0.7970051383302796}, 518: {'co': 0.7916051680097907}, 532: {'co': 0.7704140650702391}, 546: {'co': 0.7250059903394989}, 560: {'co': 0.7573655134959484}, 574: {'co': 0.7526705140844518}}
2022/07/13 20:54:28 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/07/13 20:54:46 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 16.88218929200002 seconds: {532: {'co': 0.770261719662157}, 546: {'co': 0.7266696538917392}, 560: {'co': 0.7537877960700033}, 574: {'co': 0.7628489955210572}, 588: {'co': 0.7989203481573962}, 602: {'co': 0.7800539538429004}}
2022/07/13 20:54:46 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/07/13 20:54:57 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 9.986607624999976 seconds: {5

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220628_july_gov_briefing/scenario_params.json"))

model_args = {
    'base_spec_id': 3181 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.5, 0.7, 0.9]: # 50%, 70% , or 90% for-COVID
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for immune_escape_mult in [1.0, 0.95, 1.05]:
            #for (weak_escape, strong_escape) in [(0.75, 0.1), (0.8, 0.2)]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},"desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity", "attrs": {'age': '0-19', 'vacc': 'shot1'}, "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5, "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5}}]
            immune_mult = [{"param": "immune_escape", "from_attrs":{"variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]}, "mults": {"2020-01-01": immune_escape_mult},  "desc": "sensitivity analysis for immune escape"}]
            scenario_model_args.append({'params_defs': scenario_params + school_contact + lt5_vacc_adjust + immune_mult,
                                       'hosp_reporting_frac': hrf, # hospital reporting fraction is not specified in params.json, it's a separate model argument that we created (i.e. we didn't identify a "param" in the dictionary)
                                       'tags': {'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase, 'immune_escape_mult': immune_escape_mult}})
            
            

In [ ]:
# determine the number of parameters used
len(scenario_model_args)

In [8]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args[5:15], fit_args=model_fit_args, multiprocess=multiprocess)

2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:53 INFO|============================================================
2022/07/14 10:53:57 INFO|-----------|{'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.05} Retrieving vaccinations data
2022/07/14 10:53:57 INFO|-----------|{'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.05} Retrieving vaccinations data
2022/07/14 10:53:57 INFO|-----------|{'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.0} Retrieving vaccinations data
2022/07/14 10:53:57 INFO|-----------|{'hosp_frac':

2022/07/14 10:55:17 INFO|--------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.95, 'fit_batch': '0'}: Transmission control fit 1/2 completed in 14.491341290999998 seconds: {767: {'co': 0.7898958957440059}, 781: {'co': 0.8321918827735049}, 795: {'co': 0.7994607459422483}, 809: {'co': 0.8135326198527058}, 823: {'co': 0.8179462951442315}, 837: {'co': 0.8123119853661708}}
2022/07/14 10:55:17 INFO|---------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.95, 'fit_batch': '0'}: Running forward sim
2022/07/14 10:55:17 INFO|--------|{'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.05, 'fit_batch': '0'}: Transmission control fit 1/2 completed in 14.122429666999992 seconds: {767: {'co': 0.7709366822025148}, 781: {'co': 0.8174458240126214}, 795: {'co': 0.7964273206586061}, 809: {'co': 0.8048892849023862}, 823: {'co': 0.8060687022842481}, 837: {'co': 0.795268864933497}}
2022/07/14 10:55:17 INFO|---------|{'hosp_frac

2022/07/14 10:55:33 INFO|--------|{'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.05, 'fit_batch': '1', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.8241103755501108}, 140: {'co': 0.7606632825445033}, 154: {'co': 0.6711880168053298}, 168: {'co': 0.8430307682247831}, 182: {'co': 0.7826561801378445}, 196: {'co': 0.8191830543380944}, 210: {'co': 0.7726034259026388}, 224: {'co': 0.7713676622018327}, 238: {'co': 0.7185596188624429}, 252: {'co': 0.7222209085605668}, 266: {'co': 0.6848822730370463}, 280: {'co': 0.7199443312231306}, 294: {'co': 0.7575590209953923}, 308: {'co': 0.8226615649781124}, 322: {'co': 0.8113711381077103}, 336: {'co': 0.7825940193755743}, 350: {'co': 0.7952

2022/07/14 10:55:38 INFO|-----------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.05} Constructing vaccination projections
2022/07/14 10:55:38 INFO|-----------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.05} Retrieving hospitalizations data
2022/07/14 10:55:39 INFO|============================================================
2022/07/14 10:55:39 INFO|--------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.05} Prepping Model
2022/07/14 10:55:39 INFO|---------|{'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.05} Prepping Model
2022/07/14 10:55:40 INFO|============================================================
2022/07/14 10:55:40 INFO|-----------|{'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0} Retrieving vaccinations data
2022/07/14 10:55:41 INFO|-----------|{'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.95} Retrievin

2022/07/14 10:57:15 INFO|--------|{'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1'}: Transmission control fit 2/2 completed in 14.794246084000008 seconds: {795: {'co': 0.7990805991457455}, 809: {'co': 0.8151610160717613}, 823: {'co': 0.8155688358348321}, 837: {'co': 0.8184050007634011}, 851: {'co': 0.8119412279051503}, 865: {'co': 0.8361217655145827}}
2022/07/14 10:57:15 INFO|---------|{'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1'}: Running forward sim
2022/07/14 10:57:16 INFO|--------|{'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.95, 'fit_batch': '1'}: Transmission control fit 2/2 completed in 14.681004875000013 seconds: {795: {'co': 0.7990835583503063}, 809: {'co': 0.8151558518502146}, 823: {'co': 0.815549811824867}, 837: {'co': 0.8180703596478879}, 851: {'co': 0.8114230170708714}, 865: {'co': 0.8347553573158651}}
2022/07/14 10:57:16 INFO|---------|{'hosp_frac':

2022/07/14 11:20:54 INFO|-----------------------|spec_ids: 3187,3188,3189,3190,3191,3192,3193,3194,3195,3196
2022/07/14 11:20:54 INFO|-----------------------|result_ids: 886,887,888,889,890,891,892,893,894,895


### Run the report for each fit model

In [ ]:
#spec_ids = [3187, 3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196]
#results_ids = [886, 887, 888, 889, 890, 891, 892, 893, 894, 895]

#models = []

#for spec_id in spec_ids:
    #model = CovidModel(base_spec_id = spec_id)
    #model.prep(outdir = outdir)
    #or you can say prep_model=True down below
    #models.append(model)

In [6]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

2022/07/14 09:47:44 INFO|============================================================
2022/07/14 09:47:44 INFO|--------|Solving model
2022/07/14 09:47:44 INFO|============================================================
2022/07/14 09:47:44 INFO|--------|Solving model
2022/07/14 09:47:45 INFO|============================================================
2022/07/14 09:47:45 INFO|--------|Solving model
2022/07/14 09:47:46 INFO|============================================================
2022/07/14 09:47:46 INFO|--------|Solving model
2022/07/14 09:47:47 INFO|============================================================
2022/07/14 09:47:47 INFO|--------|Solving model
2022/07/14 09:47:47 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 09:47:47 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored wh

In [7]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/07/14 09:49:25 INFO|Projecting
2022/07/14 09:49:25 INFO|
2022/07/14 09:49:40 INFO|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.0, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/14 09:49:41 INFO|
2022/07/14 09:49:56 INFO|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.95, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/14 09:49:58 INFO|
2022/07/14 09:50:12 INFO|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.05, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/14 09:50:14 INFO|
2022/07/14 09:50:29 INFO|{'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/14 09:50:30 INFO|
2022/07/14 09:50:46 INFO|{'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.95, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/14 09:50:47 INFO|R